# Understanding Vision-Language-Action Models (VLAs)

### Lab Table of Contents
* Part 1
    1. [1_imitation_learning.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part_1/1_imitation_learning.ipynb)
* **Part 2**
    1. [1_chatgpt.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/1_chatgpt.ipynb)
    2. [2_CLIP.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/2_CLIP.ipynb)
    3. [3_VLM_BLIP.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/3_VLM_BLIP.ipynb)
    4. **[4_VLA.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/4_VLA.ipynb)**
    5. [5_safety.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/5_safety.ipynb)
* [Lab Checkoff](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/checkoff.md)

### What is a VLA?

A **Vision-Language-Action (VLA)** model is the bridge between perception, language understanding, and robot control.

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Camera    │     │  Instruction│     │ Robot State │
│   Image     │     │   "pick up  │     │  (joints,   │
│             │     │   the red   │     │   gripper)  │
│             │     │   block"    │     │             │
└──────┬──────┘     └──────┬──────┘     └──────┬──────┘
       │                   │                   │
       └───────────────────┼───────────────────┘
                           │
                    ┌──────▼──────┐
                    │     VLA     │
                    │    Model    │
                    └──────┬──────┘
                           │
                    ┌──────▼──────┐
                    │   Action    │
                    │  Commands   │
                    │ (move arm,  │
                    │  close grip)│
                    └─────────────┘
```

### Key Difference from VLMs

| VLM (Vision-Language Model) | VLA (Vision-Language-Action Model) |
|-----------------------------|------------------------------------|  
| Input: Image + Text | Input: Image + Text + Robot State |
| Output: **Text** | Output: **Actions** |
| Example: "This is a cat" | Example: `[0.1, -0.3, 0.5, 0.2, 0.0, 0.1, 1.0]` |

### Learning Objectives

By the end of this notebook, you will:
1. Understand the architecture of a VLA model
2. Build a simplified VLA from scratch
3. Understand how vision, language, and state are fused
4. Learn about action tokens vs. continuous actions
5. Experiment with different model configurations

#### Ensure you have a new environment Set-Up:
* `conda create -n <env_name> python=3.10`

In [ ]:

# Install dependencies and import educational VLA module

!pip install torch numpy matplotlib --quiet


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Import our VLA module
from VLA_exercises import (
    SimpleVLA, 
    VisionEncoder, 
    LanguageEncoder, 
    StateEncoder,
    MultimodalFusion,
    ActionDecoder,
    SimpleTokenizer,
    create_demo_inputs,
    run_demo
)

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✓ All modules loaded successfully!")


### Run the quick demo below to see what a VLA does!


In [ ]:
# Run the demo from our module
model, output = run_demo()


## Understanding Each Component

A VLA has three main input encoders. Let's examine each one.

### 1. Vision Encoder

The vision encoder converts camera images into a feature vector that captures the visual scene.

**Real VLAs use:** ViT (Vision Transformer), SigLIP, CLIP visual encoder  
**Our educational version uses:** A simple CNN


In [ ]:
vision_encoder = VisionEncoder(input_channels=3, embedding_dim=256)

sample_image = torch.randn(1, 3, 224, 224)

print(f"Input image shape: {sample_image.shape}")
print(f"  - Batch size: {sample_image.shape[0]}")
print(f"  - Channels (RGB): {sample_image.shape[1]}")
print(f"  - Height: {sample_image.shape[2]}")
print(f"  - Width: {sample_image.shape[3]}")

# Encode the image
vision_embedding = vision_encoder(sample_image)

print(f"\nOutput embedding shape: {vision_embedding.shape}")
print(f"  - The 224x224x3 image is now a 256-dimensional vector!")


### Trace the Dimensions

Add print statements to understand how the image dimensions change through the CNN layers.

1. Why does the image get smaller as it passes through convolutional layers?


In [ ]:
# EXERCISE 1: Trace dimensions through the vision encoder

class VisionEncoderDebug(nn.Module):
    """Vision encoder with debug print statements."""
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((4, 4))
        self.fc = nn.Linear(128 * 4 * 4, 256)
    
    def forward(self, image):
        print(f"Input: {image.shape}")
        
        x = torch.relu(self.conv1(image))
        print(f"After conv1 (8x8 kernel, stride 4): {x.shape}")
        
        # TODO: Fill in the correct shape after conv2
        x = torch.relu(self.conv2(x))
        print(f"After conv2: {x.shape}")
        
        x = torch.relu(self.conv3(x))
        print(f"After conv3: {x.shape}")
        
        x = self.pool(x)
        print(f"After pooling: {x.shape}")
        
        x = x.view(x.size(0), -1)
        print(f"After flatten: {x.shape}")
        
        x = self.fc(x)
        print(f"Final embedding: {x.shape}")
        
        return x

# Test your debug encoder
debug_encoder = VisionEncoderDebug()
_ = debug_encoder(sample_image)


### 2. Language Encoder

The language encoder converts text instructions into a feature vector.

**Real VLAs use:** Pretrained LLMs (Llama, SmolLM), CLIP text encoder  
**Our educational version uses:** Word embeddings + LSTM


In [ ]:
# Create a tokenizer and language encoder
tokenizer = SimpleTokenizer(vocab_size=1000)
language_encoder = LanguageEncoder(vocab_size=1000, embedding_dim=256)

# Sample instructions
instructions = [
    "pick up the red block",
    "move the cup to the left",
    "push the button"
]

print("Tokenizing instructions:\n")
for instruction in instructions:
    tokens = tokenizer.encode(instruction)
    print(f"'{instruction}'")
    print(f"  → Tokens: {tokens[:10].tolist()}... (padded to 32)")
    print()


In [ ]:
# Encode the first instruction
instruction = "pick up the red block"
tokens = tokenizer.encode(instruction).unsqueeze(0)  # Add batch dimension

print(f"Token shape: {tokens.shape}")

language_embedding = language_encoder(tokens)

print(f"Language embedding shape: {language_embedding.shape}")
print(f"  - The instruction is now a 256-dimensional vector!")


### 3. State Encoder

The state encoder processes the robot's current configuration (joint positions, gripper state, etc.).

For a typical 6-DOF robot arm:
- 6 joint angles (shoulder, elbow, wrist, etc.)
- 1 gripper state (open/closed)


In [ ]:
# Create a state encoder
state_encoder = StateEncoder(state_dim=7, embedding_dim=256)

# Sample robot state: [joint1, joint2, joint3, joint4, joint5, joint6, gripper]
robot_state = torch.tensor([[
    0.5,   # Joint 1 angle (normalized)
    -0.3,  # Joint 2 angle
    0.8,   # Joint 3 angle
    0.0,   # Joint 4 angle
    0.2,   # Joint 5 angle
    -0.1,  # Joint 6 angle
    1.0    # Gripper (1.0 = closed, -1.0 = open)
]])

print(f"Robot state: {robot_state.tolist()[0]}")
print(f"State shape: {robot_state.shape}")

state_embedding = state_encoder(robot_state)

print(f"\nState embedding shape: {state_embedding.shape}")
print(f"  - The 7 joint values are now a 256-dimensional vector!")


## Multimodal Fusion

**The key insight of VLAs:** How do we combine vision, language, and state into a unified representation?

This is one of the most important design decisions in VLA architecture.


In [ ]:
# We have three embeddings of the same dimension
print("Input embeddings:")
print(f"  Vision:   {vision_embedding.shape}")
print(f"  Language: {language_embedding.shape}")
print(f"  State:    {state_embedding.shape}")

# Create fusion modules with different strategies
fusion_concat = MultimodalFusion(embedding_dim=256, fusion_dim=512, fusion_type="concat")
fusion_attention = MultimodalFusion(embedding_dim=256, fusion_dim=512, fusion_type="attention")

# Try concatenation fusion
fused_concat = fusion_concat(vision_embedding, language_embedding, state_embedding)
print(f"\nConcatenation fusion output: {fused_concat.shape}")

# Try attention fusion
fused_attention = fusion_attention(vision_embedding, language_embedding, state_embedding)
print(f"Attention fusion output: {fused_attention.shape}")


### Implement Multiplicative Fusion

2. Implement a new fusion strategy that multiplies embeddings element-wise.

| Fusion Type | Pros | Cons |
|-------------|------|------|
| Concatenation | Simple, fast | Doesn't capture interactions |
| Attention | Learns modality importance | More complex |
| Multiplicative | Forces shared representations | FILL IN! |


In [ ]:
# Implement multiplicative fusion

class MultiplicativeFusion(nn.Module):
    """Fuses embeddings by element-wise multiplication."""
    
    def __init__(self, embedding_dim=256, fusion_dim=512):
        super().__init__()
        # TODO: Define the layers needed
        # Hint: After multiplying, you need to project to fusion_dim
        self.projection = nn.Linear(embedding_dim, fusion_dim)  # Fix this if needed
    
    def forward(self, vision_emb, language_emb, state_emb):
        # TODO: Implement multiplicative fusion
        # Step 1: Multiply all three embeddings element-wise
        # Step 2: Project to fusion_dim
        
        # Your code here:
        # fused = 
        # return self.projection(fused)
        pass

# Test your implementation (uncomment when ready)
# mult_fusion = MultiplicativeFusion()
# fused_mult = mult_fusion(vision_embedding, language_embedding, state_embedding)
# print(f"Multiplicative fusion output: {fused_mult.shape}")


## Action Decoding

The final stage: converting the fused representation into robot actions.

### Two Approaches:

1. **Continuous Actions** (Traditional robotics)
   - Output: `[-0.3, 0.5, 0.1, ...]` (float values)
   - Pro: Precise control
   - Con: Hard to use language model techniques

2. **Action Tokens** (Modern VLAs like RT-2)
   - Output: `[127, 192, 64, ...]` (discrete bins)
   - Pro: Can use autoregressive generation like LLMs
   - Con: Quantization loses some precision


In [ ]:
# Create action decoders with both approaches
decoder_continuous = ActionDecoder(fusion_dim=512, action_dim=7, use_action_tokens=False)
decoder_tokens = ActionDecoder(fusion_dim=512, action_dim=7, num_action_bins=256, use_action_tokens=True)

print("Fused input shape:", fused_attention.shape)
print()

# Continuous actions
output_continuous = decoder_continuous(fused_attention)
print("=== Continuous Action Output ===")
print(f"Actions shape: {output_continuous['actions'].shape}")
print(f"Actions: {output_continuous['actions'][0].detach().numpy().round(3)}")
print()

# Tokenized actions
output_tokens = decoder_tokens(fused_attention)
print("=== Tokenized Action Output ===")
print(f"Action tokens: {output_tokens['action_tokens'][0].tolist()}")
print(f"Converted to continuous: {output_tokens['actions'][0].detach().numpy().round(3)}")


### Calculate Action Token Precision

If we discretize actions into 256 bins:
1. What is the precision (step size) between bins?
2. Is this precise enough for delicate robot manipulation?


In [ ]:
# Calculate action precision

num_bins = 256
action_range = 2.0  # Actions range from -1 to 1

# TODO: Calculate the precision (step size between bins)
# Formula: action_range / (num_bins - 1)
precision_256 = action_range / (num_bins - 1)

print(f"With {num_bins} bins:")
print(f"  Action range: [-1, 1] = {action_range}")
print(f"  Precision: {precision_256:.6f}")

# TODO: What if we only had 16 bins?
num_bins_16 = 16
precision_16 = action_range / (num_bins_16 - 1)
print(f"\nWith only {num_bins_16} bins:")
print(f"  Precision: {precision_16:.6f}")

# Reflection: A typical servo motor has ~0.1 degree precision
# Is 256 bins enough?


## The Complete VLA Pipeline

Now let's use the complete SimpleVLA model and see the full pipeline in action.


In [ ]:
# Create the complete VLA model
vla = SimpleVLA(
    image_channels=3,
    vocab_size=1000,
    state_dim=7,
    action_dim=7,
    embedding_dim=256,
    fusion_dim=512,
    num_action_bins=256,
    fusion_type="attention",
    use_action_tokens=True
)

vla.eval()
print(f"Model created with {sum(p.numel() for p in vla.parameters()):,} parameters")


In [ ]:
# Create inputs
image = torch.randn(1, 3, 224, 224)  # Random image
instruction = "pick up the red block"
tokens = tokenizer.encode(instruction).unsqueeze(0)
state = torch.tensor([[0.5, -0.3, 0.8, 0.0, 0.2, -0.1, 1.0]])

print("Inputs:")
print(f"  Image: {image.shape}")
print(f"  Instruction: '{instruction}'")
print(f"  Robot state: {state[0].tolist()}")

# Forward pass
with torch.no_grad():
    output = vla(image, tokens, state)

print("\n" + "="*50)
print("OUTPUT ACTIONS")
print("="*50)

actions = output['actions'][0].numpy()
action_names = ["joint_1", "joint_2", "joint_3", "joint_4", "joint_5", "joint_6", "gripper"]

for name, value in zip(action_names, actions):
    bar_len = int((value + 1) * 20)
    bar = "█" * max(0, bar_len)
    print(f"  {name:10s}: {value:+.3f} |{bar}")


### Test Instruction Sensitivity

1. Does the (untrained) model respond differently to different instructions?

**Key insight:** An *untrained* model produces random outputs. Only through training does it learn meaningful mappings!


In [ ]:
# Test different instructions

test_instructions = [
    "pick up the red block",
    "pick up the blue block", 
    "put down the object",
    "move left",
    "wave hello"
]

print("Testing instruction sensitivity:\n")
print("Note: This is an UNTRAINED model!")
print("="*60)

results = []

for test_instruction in test_instructions:
    test_tokens = tokenizer.encode(test_instruction).unsqueeze(0)
    
    with torch.no_grad():
        test_output = vla(image, test_tokens, state)
    
    test_actions = test_output['actions'][0].numpy()
    results.append(test_actions)
    
    print(f"\n'{test_instruction}'")
    print(f"  Actions: {test_actions.round(2)}")

print("\n" + "="*60)
print("Similarity analysis (lower = more similar):")
for i in range(len(test_instructions)):
    for j in range(i+1, len(test_instructions)):
        diff = np.abs(results[i] - results[j]).mean()
        print(f"  '{test_instructions[i][:15]}' vs '{test_instructions[j][:15]}': {diff:.3f}")


## Visualizing the VLA Pipeline


In [ ]:
def visualize_vla_pipeline(model, img, toks, st, instr):
    """Visualize the VLA pipeline with embeddings and outputs."""
    with torch.no_grad():
        out = model(img, toks, st)
    
    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    fig.suptitle(f"VLA Pipeline: '{instr}'", fontsize=14, fontweight='bold')
    
    # Vision embedding
    ax = axes[0, 0]
    vision_emb = out['vision_embedding'][0].numpy()
    ax.bar(range(50), vision_emb[:50], color='#FF6B6B', alpha=0.7)
    ax.set_title('Vision Embedding', fontweight='bold')
    ax.set_xlabel('Dimension')
    
    # Language embedding  
    ax = axes[0, 1]
    lang_emb = out['language_embedding'][0].numpy()
    ax.bar(range(50), lang_emb[:50], color='#4ECDC4', alpha=0.7)
    ax.set_title('Language Embedding', fontweight='bold')
    ax.set_xlabel('Dimension')
    
    # State embedding
    ax = axes[0, 2]
    state_emb = out['state_embedding'][0].numpy()
    ax.bar(range(50), state_emb[:50], color='#45B7D1', alpha=0.7)
    ax.set_title('State Embedding', fontweight='bold')
    ax.set_xlabel('Dimension')
    
    # Fused embedding
    ax = axes[1, 0]
    fused_emb = out['fused_embedding'][0].numpy()
    ax.bar(range(100), fused_emb[:100], color='#9B59B6', alpha=0.7)
    ax.set_title('Fused Embedding', fontweight='bold')
    ax.set_xlabel('Dimension')
    
    # Actions
    ax = axes[1, 1]
    act = out['actions'][0].numpy()
    names = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'Grip']
    colors = ['#E74C3C' if a < 0 else '#27AE60' for a in act]
    ax.barh(names, act, color=colors, alpha=0.7)
    ax.set_title('Output Actions', fontweight='bold')
    ax.set_xlim(-1.5, 1.5)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    
    # Architecture diagram
    ax = axes[1, 2]
    ax.text(0.5, 0.8, 'VLA Architecture', ha='center', fontsize=12, fontweight='bold')
    ax.text(0.5, 0.6, 'Vision + Language + State', ha='center', fontsize=10)
    ax.text(0.5, 0.45, '↓', ha='center', fontsize=14)
    ax.text(0.5, 0.3, 'Fusion', ha='center', fontsize=10)
    ax.text(0.5, 0.15, '↓', ha='center', fontsize=14)
    ax.text(0.5, 0.0, 'Action Tokens', ha='center', fontsize=10)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_vla_pipeline(vla, image, tokens, state, instruction)


## More Reflection Questions

Before moving on, answer these questions with your lab partner:

1. **Architecture Design:** Why do we convert all modalities to the same embedding dimension before fusion?

2. **Training Data:** What kind of data would you need to train a VLA? How is this different from training a VLM? What are the potential challenges and safety concerns?



## OPTIONAL Challenge Exercises

### Challenge 1: Temporal Vision

Real robots need to consider history. Modify the vision encoder to process a sequence of frames.


In [ ]:
# CHALLENGE 1: Temporal Vision Encoder

class TemporalVisionEncoder(nn.Module):
    """
    Input: (batch, num_frames, channels, height, width)
    Output: (batch, embedding_dim)
    """
    def __init__(self, embedding_dim=256, num_frames=4):
        super().__init__()
        # TODO: Implement
        # Hint: Process each frame with CNN, then use LSTM across time
        pass
    
    def forward(self, image_sequence):
        # TODO: Implement
        pass

# Test (uncomment when ready)
# temporal_encoder = TemporalVisionEncoder()
# image_seq = torch.randn(1, 4, 3, 224, 224)
# temporal_emb = temporal_encoder(image_seq)
# print(f"Temporal embedding: {temporal_emb.shape}")


### Challenge 2: Action Chunking

Instead of predicting one action, predict a sequence of K future actions.


In [ ]:
# CHALLENGE 2: Action Chunking

class ChunkedActionDecoder(nn.Module):
    """
    Input: (batch, fusion_dim)
    Output: (batch, chunk_size, action_dim)
    """
    def __init__(self, fusion_dim=512, action_dim=7, chunk_size=10):
        super().__init__()
        self.chunk_size = chunk_size
        self.action_dim = action_dim
        # TODO: Implement
        pass
    
    def forward(self, fused):
        # TODO: Implement
        pass

# Test (uncomment when ready)
# chunked = ChunkedActionDecoder(chunk_size=10)
# action_chunk = chunked(fused_attention)
# print(f"Action chunk: {action_chunk.shape}")  # Should be (1, 10, 7)


### Continue to
[5_safety.ipynb](https://github.com/abbykoneill/lerobot/blob/main/lab_part2/5_safety.ipynb)

## References

* [RT-2: Vision-Language-Action Models](https://robotics-transformer2.github.io/)
* [OpenVLA](https://openvla.github.io/)
* [SmolVLA](https://huggingface.co/blog/smolvla)
* [LeRobot](https://github.com/huggingface/lerobot)